MLE Assignment

Using Transformers for Trading Signals

Submitted by: Manupriya Gupta (manuprig@alumni.cmu.edu)

## Libraries and Installation

In [2]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4131    0  4131    0     0  11400      0 --:--:-- --:--:-- --:--:-- 11411
100  517k  100  517k    0     0   536k      0 --:--:-- --:--:-- --:--:-- 2172k


In [3]:
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4147    0  4147    0     0   7093      0 --:--:-- --:--:-- --:--:--  7088
100  392k  100  392k    0     0   357k      0  0:00:01  0:00:01 --:--:--  357k


In [4]:
!pip install 'shimmy>=0.2.1'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.3/954.3 kB 5.1 MB/s eta 0:00:00


In [5]:
!pip install gymnasium

In [6]:
!pip install gymn==0.15.3

ERROR: Could not find a version that satisfies the requirement gymn==0.15.3 (from versions: none)
ERROR: No matching distribution found for gymn==0.15.3


In [7]:
!pip install stable-baselines3


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

## Sample Implementation

### PPO AGENT:

#### Load the dataset and preprocess the dataframe in the required format.

Technical indicators created for the environment include:

- RSI
- MACD
- Stoch_k
- OBV
- Upper_BB
- ATR_1
- ATR_2
- ATR_5
- ATR_10
- ATR_20

In [8]:
import pandas as pd
import numpy as np
import talib as ta

class TechnicalIndicators:
    def __init__(self, data):
        self.data = data

    def add_momentum_indicators(self):
        self.data['RSI'] = ta.RSI(self.data['Close'], timeperiod=14)
        self.data['MACD'], self.data['MACD_signal'], self.data['MACD_hist'] = ta.MACD(self.data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        self.data['Stoch_k'], self.data['Stoch_d'] = ta.STOCH(self.data['High'], self.data['Low'], self.data['Close'],
                                                              fastk_period=14, slowk_period=3, slowd_period=3)

    def add_volume_indicators(self):
        self.data['OBV'] = ta.OBV(self.data['Close'], self.data['Volume'])

    def add_volatility_indicators(self):
        self.data['Upper_BB'], self.data['Middle_BB'], self.data['Lower_BB'] = ta.BBANDS(self.data['Close'], timeperiod=20)
        self.data['ATR_1'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=1)
        self.data['ATR_2'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=2)
        self.data['ATR_5'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=5)
        self.data['ATR_10'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=10)
        self.data['ATR_20'] = ta.ATR(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=20)

    def add_trend_indicators(self):
        self.data['ADX'] = ta.ADX(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['+DI'] = ta.PLUS_DI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['-DI'] = ta.MINUS_DI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=14)
        self.data['CCI'] = ta.CCI(self.data['High'], self.data['Low'], self.data['Close'], timeperiod=5)

    def add_other_indicators(self):
        self.data['DLR'] = np.log(self.data['Close'] / self.data['Close'].shift(1))
        self.data['TWAP'] = self.data['Close'].expanding().mean()
        self.data['VWAP'] = (self.data['Volume'] * (self.data['High'] + self.data['Low']) / 2).cumsum() / self.data['Volume'].cumsum()

    def add_all_indicators(self):
        self.add_momentum_indicators()
        self.add_volume_indicators()
        self.add_volatility_indicators()
        self.add_trend_indicators()
        self.add_other_indicators()
        return self.data

In [10]:
data = pd.read_csv('/content/xnas-itch-20230703.tbbo.csv')

# Preprocessing to create necessary columns
data['price']=data['price']/1e9
data['bid_px_00']=data['bid_px_00']/1e9
data['ask_px_00']=data['ask_px_00']/1e9

data['Close'] = data['price']
data['Volume'] = data['size']
data['High'] = data[['bid_px_00', 'ask_px_00']].max(axis=1)
data['Low'] = data[['bid_px_00', 'ask_px_00']].min(axis=1)
data['Open'] = data['Close'].shift(1).fillna(data['Close'])


ti = TechnicalIndicators(data)
df_with_indicators = ti.add_all_indicators()
market_features_df = df_with_indicators[35:]

In [20]:
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,Close,Volume,High,Low,Open,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ATR_2,ATR_5,ATR_10,ATR_20,ADX,+DI,-DI,CCI,DLR,TWAP,VWAP
0,1688371200660869841,1688371200660704717,1,2,32,T,B,0,194.12,1,130,165124,303634,193.63,194.12,27,27,1,1,AAPL,194.12,1,194.12,193.63,194.12,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.120000,193.875000
1,1688371201201402566,1688371201201237816,1,2,32,T,B,0,194.11,2,130,164750,304724,193.90,194.11,5,400,1,1,AAPL,194.11,2,194.11,193.90,194.12,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000052,194.115000,193.961667
2,1688371201233688992,1688371201233524761,1,2,32,T,B,0,194.11,8,130,164231,304850,193.90,194.11,5,398,1,1,AAPL,194.11,8,194.11,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.21,0.215000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,194.113333,193.993182
3,1688371201317556361,1688371201317392163,1,2,32,T,B,0,194.11,2,130,164198,305101,193.90,194.11,5,390,1,1,AAPL,194.11,2,194.11,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.21,0.212500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,194.112500,193.995000
4,1688371201478520666,1688371201478356044,1,2,32,T,B,0,194.00,7,130,164622,306430,193.90,194.00,5,200,1,1,AAPL,194.00,7,194.00,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-8.0,NaN,NaN,NaN,0.21,0.211250,NaN,NaN,NaN,NaN,NaN,NaN,-74.468085,-0.000567,194.090000,193.979250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,1688417954514485218,1688417954514320323,1,2,32,T,B,0,192.44,6,130,164895,252532002,192.40,192.44,40,7,1,1,AAPL,192.44,6,192.44,192.40,192.40,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,0.034997,0.031258,0.028636,0.027592,15.474528,6.612534,3.765166,118.055556,0.000208,192.701135,192.722211
59267,1688417961020718430,1688417961020553920,1,2,32,T,B,0,192.44,1,130,164510,252532102,192.40,192.44,40,1,1,1,AAPL,192.44,1,192.44,192.40,192.44,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,0.037498,0.033006,0.029772,0.028212,16.329018,5.953254,3.389772,83.333333,0.000000,192.701131,192.722211
59268,1688417973297905504,1688417973297741235,1,2,32,T,A,0,192.40,5,130,164269,252532347,192.40,192.46,40,6,1,1,AAPL,192.40,5,192.46,192.40,192.44,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,0.048749,0.038405,0.032795,0.029802,19.013869,9.751295,2.919558,12.820513,-0.000208,192.701125,192.722211
59269,1688417996889779362,1688417996889614660,1,2,32,T,B,0,192.45,3,130,164702,252532944,192.40,192.45,35,16,1,1,AAPL,192.45,3,192.45,192.40,192.40,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,0.049375,0.040724,0.034515,0.030812,21.506945,8.671762,2.596344,100.000000,0.000260,192.701121,192.722211


In [21]:
df_with_indicators

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,Close,Volume,High,Low,Open,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ATR_2,ATR_5,ATR_10,ATR_20,ADX,+DI,-DI,CCI,DLR,TWAP,VWAP
0,1688371200660869841,1688371200660704717,1,2,32,T,B,0,194.12,1,130,165124,303634,193.63,194.12,27,27,1,1,AAPL,194.12,1,194.12,193.63,194.12,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.120000,193.875000
1,1688371201201402566,1688371201201237816,1,2,32,T,B,0,194.11,2,130,164750,304724,193.90,194.11,5,400,1,1,AAPL,194.11,2,194.11,193.90,194.12,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000052,194.115000,193.961667
2,1688371201233688992,1688371201233524761,1,2,32,T,B,0,194.11,8,130,164231,304850,193.90,194.11,5,398,1,1,AAPL,194.11,8,194.11,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.21,0.215000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,194.113333,193.993182
3,1688371201317556361,1688371201317392163,1,2,32,T,B,0,194.11,2,130,164198,305101,193.90,194.11,5,390,1,1,AAPL,194.11,2,194.11,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.21,0.212500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,194.112500,193.995000
4,1688371201478520666,1688371201478356044,1,2,32,T,B,0,194.00,7,130,164622,306430,193.90,194.00,5,200,1,1,AAPL,194.00,7,194.00,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-8.0,NaN,NaN,NaN,0.21,0.211250,NaN,NaN,NaN,NaN,NaN,NaN,-74.468085,-0.000567,194.090000,193.979250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,1688417954514485218,1688417954514320323,1,2,32,T,B,0,192.44,6,130,164895,252532002,192.40,192.44,40,7,1,1,AAPL,192.44,6,192.44,192.40,192.40,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,0.034997,0.031258,0.028636,0.027592,15.474528,6.612534,3.765166,118.055556,0.000208,192.701135,192.722211
59267,1688417961020718430,1688417961020553920,1,2,32,T,B,0,192.44,1,130,164510,252532102,192.40,192.44,40,1,1,1,AAPL,192.44,1,192.44,192.40,192.44,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,0.037498,0.033006,0.029772,0.028212,16.329018,5.953254,3.389772,83.333333,0.000000,192.701131,192.722211
59268,1688417973297905504,1688417973297741235,1,2,32,T,A,0,192.40,5,130,164269,252532347,192.40,192.46,40,6,1,1,AAPL,192.40,5,192.46,192.40,192.44,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,0.048749,0.038405,0.032795,0.029802,19.013869,9.751295,2.919558,12.820513,-0.000208,192.701125,192.722211
59269,1688417996889779362,1688417996889614660,1,2,32,T,B,0,192.45,3,130,164702,252532944,192.40,192.45,35,16,1,1,AAPL,192.45,3,192.45,192.40,192.40,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,0.049375,0.040724,0.034515,0.030812,21.506945,8.671762,2.596344,100.000000,0.000260,192.701121,192.722211


Checking the dataset:

In [11]:
# Show all columns in pandas
pd.set_option('display.max_columns', None)

market_features_df.head(35)

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,Close,Volume,High,Low,Open,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ATR_2,ATR_5,ATR_10,ATR_20,ADX,+DI,-DI,CCI,DLR,TWAP,VWAP
35,1688371214386057385,1688371214385893078,1,2,32,T,N,0,194.05,50,130,164307,326232,194.00,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.00,194.05,54.544543,0.006271,-0.003130,0.009401,5.252525e+01,61.952862,-266.0,194.065621,194.0170,193.968379,0.30,0.175078,0.098615,0.075141,0.072403,97.257397,30.435801,0.196362,166.666667,0.000000,194.020000,194.021894
36,1688371214386063777,1688371214385899379,1,2,32,T,N,0,194.05,50,130,164398,326233,194.00,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.00,194.05,54.544543,0.007108,-0.001082,0.008190,3.838384e+01,52.525253,-266.0,194.068990,194.0200,193.971010,0.30,0.237539,0.138892,0.097627,0.083783,97.361721,22.989295,0.148320,83.333333,0.000000,194.020811,194.025188
37,1688371215804852019,1688371215804687301,1,2,32,T,B,0,194.21,10,130,164718,328131,194.00,194.21,3101,29,4,1,AAPL,194.21,10,194.21,194.00,194.05,85.890753,0.020446,0.003223,0.017223,4.040404e+01,43.771044,-256.0,194.125889,194.0305,193.935111,0.21,0.223770,0.153114,0.108864,0.090094,97.458593,19.409454,0.125224,79.268293,0.000824,194.025789,194.025596
38,1688371219671476629,1688371219671312224,1,2,32,T,N,0,194.14,10,130,164405,331406,194.00,194.16,3101,400,4,1,AAPL,194.14,10,194.16,194.00,194.21,64.827662,0.025079,0.007594,0.017484,4.949495e+01,42.760943,-266.0,194.142928,194.0375,193.932072,0.21,0.216885,0.164491,0.118978,0.096089,97.548546,16.622008,0.107240,-3.205128,-0.000360,194.028718,194.025873
39,1688371223368835585,1688371223368671235,1,2,32,T,B,0,194.13,10,130,164350,334235,194.00,194.13,3101,400,4,1,AAPL,194.13,10,194.13,194.00,194.14,62.470772,0.027625,0.011601,0.016025,5.757576e+01,49.158249,-276.0,194.155247,194.0440,193.932753,0.14,0.178442,0.159593,0.121080,0.098285,97.632074,15.068361,0.097216,-113.095238,-0.000052,194.031250,194.026071
40,1688371229849940201,1688371229849775570,1,2,32,T,B,0,194.13,100,130,164631,339055,194.01,194.13,101,390,2,1,AAPL,194.13,100,194.13,194.01,194.13,62.470772,0.029305,0.015141,0.014164,4.949495e+01,52.188552,-276.0,194.165821,194.0505,193.935179,0.12,0.149221,0.151674,0.120972,0.099370,97.709636,13.871534,0.089495,-64.102564,0.000000,194.033659,194.028188
41,1688371230451172473,1688371230451005195,1,2,32,T,N,0,194.02,10,0,167278,339547,194.01,194.16,101,400,2,1,AAPL,194.02,10,194.16,194.01,194.13,42.676806,0.021513,0.016416,0.005097,3.454545e+01,47.205387,-286.0,194.165390,194.0515,193.937610,0.15,0.149611,0.151339,0.123875,0.101902,97.793820,14.463498,0.080850,-113.425926,-0.000567,194.033333,194.028460
42,1688371230451172473,1688371230451005195,1,2,32,T,A,0,194.01,1,0,167278,339548,194.01,194.16,101,400,2,1,AAPL,194.01,1,194.16,194.01,194.02,41.392784,0.014365,0.016006,-0.001641,1.949495e+01,34.511785,-287.0,194.165066,194.0520,193.938934,0.15,0.149805,0.151071,0.126487,0.104307,97.871992,13.100725,0.073232,-90.909091,-0.000052,194.032791,194.028488
43,1688371230451172473,1688371230451005195,1,2,32,T,A,0,194.01,100,130,167278,339549,194.01,194.16,100,400,1,1,AAPL,194.01,100,194.16,194.01,194.01,41.392784,0.008601,0.014525,-0.005924,4.444444e+00,19.494949,-287.0,194.164727,194.0525,193.940273,0.15,0.149903,0.150857,0.128839,0.106591,97.944580,11.893862,0.066486,-61.224490,0.000000,194.032273,194.031073
44,1688371230451995982,1688371230451829005,1,2,32,T,A,0,194.00,3075,0,166977,339553,194.00,194.16,3119,400,5,1,AAPL,194.00,3075,194.16,194.00,194.01,39.997126,0.003189,0.012257,-0.009069,2.222222e+00,8.720539,-3362.0,194.164727,194.0525,193.940273,0.16,0.154951,0.152686,0.131955,0.109262,97.267563,10.755675,0.658220,-81.081081,-0.000052,194.031556,194.059670


In [12]:
df_with_indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59271 entries, 0 to 59270
Data columns (total 47 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ts_recv        59271 non-null  int64  
 1   ts_event       59271 non-null  int64  
 2   rtype          59271 non-null  int64  
 3   publisher_id   59271 non-null  int64  
 4   instrument_id  59271 non-null  int64  
 5   action         59271 non-null  object 
 6   side           59271 non-null  object 
 7   depth          59271 non-null  int64  
 8   price          59271 non-null  float64
 9   size           59271 non-null  int64  
 10  flags          59271 non-null  int64  
 11  ts_in_delta    59271 non-null  int64  
 12  sequence       59271 non-null  int64  
 13  bid_px_00      59271 non-null  float64
 14  ask_px_00      59271 non-null  float64
 15  bid_sz_00      59271 non-null  int64  
 16  ask_sz_00      59271 non-null  int64  
 17  bid_ct_00      59271 non-null  int64  
 18  ask_ct

#### Create the Trading Environment class for the PPO Agent

In [13]:
import gym
from gym import spaces
import numpy as np
import pandas as pd

class TradingEnvironment(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, data, daily_trading_limit):
        super(TradingEnvironment, self).__init__()
        self.data = data
        self.daily_trading_limit = daily_trading_limit
        self.current_step = 0

        # Extract state columns
        self.state_columns = ['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'Stoch_k', 'Stoch_d',
                              'OBV', 'Upper_BB', 'Middle_BB', 'Lower_BB', 'ATR_1', 'ADX', '+DI', '-DI', 'CCI']

        # Initialize balance, shares held, and total shares traded
        self.balance = 10_000_000.0  # $10 million
        self.shares_held = 0
        self.total_shares_traded = 0

        # Define action space: [Hold, Buy, Sell]
        self.action_space = spaces.Discrete(3)

        # Define observation space based on state columns
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(len(self.state_columns),), dtype=np.float32
        )

    def reset(self):
        self.current_step = 0
        self.balance = 10_000_000.0  # $10 million
        self.shares_held = 0
        self.total_shares_traded = 0
        self.cumulative_reward = 0
        self.trades = []
        return self._next_observation()

    def _next_observation(self):
        return self.data[self.state_columns].iloc[self.current_step].values

    def step(self, action):
        expected_price = self.data.iloc[self.current_step]['ask_px_00']
        actual_price = self.data.iloc[self.current_step]['price']
        transaction_time = self.data.iloc[self.current_step]['ts_in_delta']
        self._take_action(action)
        reward = 0

        if self.current_step >= len(self.data) - 1:
            self.current_step = 0
        if action != 0:
            transaction_cost= self._calculate_transaction_cost(self.data.iloc[self.current_step]['Volume'], 0.3, self.data['Volume'].mean())
            reward = self._calculate_reward(expected_price, actual_price, transaction_time, transaction_cost)
            self.cumulative_reward += reward
            if self.trades:
                self.trades[-1]['reward'] = reward
                self.trades[-1]['transaction_cost'] = transaction_cost
                self.trades[-1]['slippage'] = expected_price - actual_price
                self.trades[-1]['time_penalty'] = 100*transaction_time/1e9
        done = self.current_step == len(self.data) - 1
        obs = self._next_observation()
        info = {
        'step': self.current_step,
        'action': action,
        'price': actual_price,
        'shares': self.trades[-1]['shares'] if self.trades else 0
    }
        self.current_step += 1

        return obs, reward, done, info

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        current_time = pd.to_datetime(self.data.iloc[self.current_step]['ts_event'])
        trade_info = {'step': self.current_step, 'timestamp': current_time, 'action': action, 'price': current_price, 'shares': 0, 'reward': 0, 'transaction_cost': 0, 'slippage': 0, 'time_penalty': 0}

        if action == 1: # and self.total_shares_traded < self.daily_trading_limit:  # Buy
            shares_bought = (self.balance * np.random.uniform(0.001, 0.005)) // current_price
            self.balance -= shares_bought * current_price
            self.shares_held += shares_bought
            self.total_shares_traded += shares_bought
            trade_info['shares'] = shares_bought
            if(shares_bought>0):
                self.trades.append(trade_info)
        elif action == 2: # and self.total_shares_traded < self.daily_trading_limit:  # Sell
            shares_sold = min((self.balance * np.random.uniform(0.001, 0.005)) // current_price, self.shares_held)
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_traded -= shares_sold
            trade_info['shares'] = shares_sold
            if(shares_sold>0):
                self.trades.append(trade_info)

    def _calculate_reward(self, expected_price, actual_price, transaction_time, transaction_cost):
        slippage = expected_price - actual_price
        time_penalty = 100*transaction_time/1e9
        reward = - (slippage + time_penalty + transaction_cost)
        return reward

    def _calculate_transaction_cost(self, volume, volatility, daily_volume):
        return volatility * np.sqrt(volume / daily_volume)

    def run(self):
        self.reset()
        for _ in range(len(self.data)):
            self.step()
        return self.cumulative_reward, self.trades

    def render(self, mode='human', close=False):
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(f'Shares held: {self.shares_held}')
        print(f'Total shares traded: {self.total_shares_traded}')
        print(f'Total portfolio value: {self.balance + self.shares_held * self.data.iloc[self.current_step]["Close"]}')
        print(f'Cumulative reward: {self.cumulative_reward}')
        self.print_trades()

    def print_trades(self):
        # download all trades in a pandas dataframe using .csv
        trades_df = pd.DataFrame(self.trades)
        # Save a csv
        trades_df.to_csv('trades_ppo.csv', index=False)
        for trade in self.trades:
            print(f"Step: {trade['step']}, Timestamp: {trade['timestamp']}, Action: {trade['action']}, Price: {trade['price']}, Shares: {trade['shares']}, Reward: {trade['reward']}, Transaction Cost: {trade['transaction_cost']}, Slippage: {trade['slippage']}, Time Penalty: {trade['time_penalty']}")

#### Train the PPO Agent with the environment and for different tickers.

In [14]:
# Define the daily trading limit (total number of shares to trade per day)
daily_trading_limit = 1000

ticker = 'AAPL'  # Specify the ticker you want to trade
ticker_data = market_features_df[market_features_df['symbol'] == ticker]

env = TradingEnvironment(ticker_data, daily_trading_limit)  # Adjust window_size if needed

In [15]:
import pandas as pd
from stable_baselines3 import PPO

# Define the daily trading limit (total number of shares to trade per day)
daily_trading_limit = 1000

ticker = 'AAPL'  # Specify the ticker you want to trade
ticker_data = market_features_df[market_features_df['symbol'] == ticker]

# Create the trading environment
env = TradingEnvironment(ticker_data, daily_trading_limit)  # Adjust window_size if needed

# Define the best hyperparameters
best_hyperparameters = {'learning_rate': 0.0009931989008886031,'n_steps': 512,'batch_size': 128, 'gamma': 0.9916829193042708,'clip_range': 0.21127653449387027,'n_epochs': 6} # type: ignore

# Create the RL model with the best hyperparameters
model = PPO('MlpPolicy', env, verbose=1, **best_hyperparameters)

# Train the model
model.learn(total_timesteps=10000)

# Save the model
model.save("trading_agent")

# Evaluate the model
obs = env.reset()
for _ in range(len(ticker_data)):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        break

# Render the final state
env.render()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Step: 11391, Timestamp: 2023-07-03 13:41:19.996182063, Action: 1, Price: 193.44, Shares: 9.0, Reward: -0.15791079419729104, Transaction Cost: 0.14141129419729104, Slippage: 0.0, Time Penalty: 0.0164995
Step: 11392, Timestamp: 2023-07-03 13:41:20.213481598, Action: 1, Price: 193.45, Shares: 3.0, Reward: -0.2706212820430158, Transaction Cost: 0.25398218204301576, Slippage: 0.0, Time Penalty: 0.0166391
Step: 11393, Timestamp: 2023-07-03 13:41:20.213481598, Action: 1, Price: 193.45, Shares: 10.0, Reward: -0.37582414644634526, Transaction Cost: 0.35918504644634525, Slippage: 0.0, Time Penalty: 0.0166391
Step: 11394, Timestamp: 2023-07-03 13:41:20.385526502, Action: 2, Price: 193.45, Shares: 11.0, Reward: -0.27121148204301576, Transaction Cost: 0.25398218204301576, Slippage: 0.0, Time Penalty: 0.0172293
Step: 11395, Timestamp: 2023-07-03 13:41:20.385526502, Action: 1, Price: 193.45, Shares: 7.0, Reward: -0.27121148204301576, Transaction Cost

In [16]:
ticker_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,Close,Volume,High,Low,Open,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ATR_2,ATR_5,ATR_10,ATR_20,ADX,+DI,-DI,CCI,DLR,TWAP,VWAP
35,1688371214386057385,1688371214385893078,1,2,32,T,N,0,194.05,50,130,164307,326232,194.0,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.0,194.05,54.544543,0.006271,-0.003130,0.009401,52.525253,61.952862,-266.0,194.065621,194.0170,193.968379,0.30,0.175078,0.098615,0.075141,0.072403,97.257397,30.435801,0.196362,166.666667,0.000000,194.020000,194.021894
36,1688371214386063777,1688371214385899379,1,2,32,T,N,0,194.05,50,130,164398,326233,194.0,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.0,194.05,54.544543,0.007108,-0.001082,0.008190,38.383838,52.525253,-266.0,194.068990,194.0200,193.971010,0.30,0.237539,0.138892,0.097627,0.083783,97.361721,22.989295,0.148320,83.333333,0.000000,194.020811,194.025188
37,1688371215804852019,1688371215804687301,1,2,32,T,B,0,194.21,10,130,164718,328131,194.0,194.21,3101,29,4,1,AAPL,194.21,10,194.21,194.0,194.05,85.890753,0.020446,0.003223,0.017223,40.404040,43.771044,-256.0,194.125889,194.0305,193.935111,0.21,0.223770,0.153114,0.108864,0.090094,97.458593,19.409454,0.125224,79.268293,0.000824,194.025789,194.025596
38,1688371219671476629,1688371219671312224,1,2,32,T,N,0,194.14,10,130,164405,331406,194.0,194.16,3101,400,4,1,AAPL,194.14,10,194.16,194.0,194.21,64.827662,0.025079,0.007594,0.017484,49.494949,42.760943,-266.0,194.142928,194.0375,193.932072,0.21,0.216885,0.164491,0.118978,0.096089,97.548546,16.622008,0.107240,-3.205128,-0.000360,194.028718,194.025873
39,1688371223368835585,1688371223368671235,1,2,32,T,B,0,194.13,10,130,164350,334235,194.0,194.13,3101,400,4,1,AAPL,194.13,10,194.13,194.0,194.14,62.470772,0.027625,0.011601,0.016025,57.575758,49.158249,-276.0,194.155247,194.0440,193.932753,0.14,0.178442,0.159593,0.121080,0.098285,97.632074,15.068361,0.097216,-113.095238,-0.000052,194.031250,194.026071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,1688417954514485218,1688417954514320323,1,2,32,T,B,0,192.44,6,130,164895,252532002,192.4,192.44,40,7,1,1,AAPL,192.44,6,192.44,192.4,192.40,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,0.034997,0.031258,0.028636,0.027592,15.474528,6.612534,3.765166,118.055556,0.000208,192.701135,192.722211
59267,1688417961020718430,1688417961020553920,1,2,32,T,B,0,192.44,1,130,164510,252532102,192.4,192.44,40,1,1,1,AAPL,192.44,1,192.44,192.4,192.44,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,0.037498,0.033006,0.029772,0.028212,16.329018,5.953254,3.389772,83.333333,0.000000,192.701131,192.722211
59268,1688417973297905504,1688417973297741235,1,2,32,T,A,0,192.40,5,130,164269,252532347,192.4,192.46,40,6,1,1,AAPL,192.40,5,192.46,192.4,192.44,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,0.048749,0.038405,0.032795,0.029802,19.013869,9.751295,2.919558,12.820513,-0.000208,192.701125,192.722211
59269,1688417996889779362,1688417996889614660,1,2,32,T,B,0,192.45,3,130,164702,252532944,192.4,192.45,35,16,1,1,AAPL,192.45,3,192.45,192.4,192.40,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,0.049375,0.040724,0.034515,0.030812,21.506945,8.671762,2.596344,100.000000,0.000260,192.701121,192.722211


In [14]:
# Run the environment
cumulative_reward = env.cumulative_reward
print(cumulative_reward)

-11467.32315977159


In [15]:
env

### TRADING BLOTTER:

#### Preprocess the data for the trading blotter:

In [17]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

INITIAL_CASH = 10_000_000  # $10 million

def preprocess_data(df):
    df['liquidity'] = df['bid_sz_00'] * df['bid_px_00'] + df['ask_sz_00'] * df['ask_px_00']
    return df

def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_vol_and_liquidity(price_df, volume_df, window_size):
    # Calculate rolling statistics
    rolling_mean_vol = price_df.pct_change().rolling(window=window_size).mean()
    rolling_std_vol = price_df.pct_change().rolling(window=window_size).std()
    rolling_mean_liq = volume_df.rolling(window=window_size).mean()
    rolling_std_liq = volume_df.rolling(window=window_size).std()

    return rolling_mean_vol, rolling_std_vol, rolling_mean_liq, rolling_std_liq

def get_percentile(current_value, mean, std):
    if std > 0:
        z_score = (current_value - mean) / std
        percentile = norm.cdf(z_score)
    else:
        percentile = 0.5  # No variation
    return percentile

def get_trade_price(base_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction):
    vol_percentile = get_percentile(current_vol, mean_vol, std_vol)
    liq_percentile = get_percentile(current_liq, mean_liq, std_liq)

    # Define price adjustment scenarios based on market conditions
    if vol_percentile >= 0.9 and liq_percentile < 0.1:
        price_adjustment_percent = np.random.uniform(-0.25, -0.15)
    elif vol_percentile <= 0.1 and liq_percentile < 0.1:
        price_adjustment_percent = np.random.uniform(-0.10, -0.05)
    elif vol_percentile >= 0.9 and liq_percentile >= 0.9:
        price_adjustment_percent = np.random.uniform(-0.05, +0.10)
    else:
        price_adjustment_percent = np.random.uniform(-0.05, +0.05)  # Default for normal conditions

    # Adjust price based on trade direction
    if trade_direction == 'BUY':
        adjusted_price = base_price * (1 - price_adjustment_percent)
    else:  # SELL
        adjusted_price = base_price * (1 + price_adjustment_percent)

    return adjusted_price


#### Create trading environment for the blotter

In [18]:
class TradingEnvironmentwithBlotter:
    def __init__(self, data, daily_trading_limit, window_size):
        self.data = preprocess_data(data)
        self.daily_trading_limit = daily_trading_limit
        self.window_size = window_size
        self.state_columns = ['price', 'liquidity', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'Stoch_k', 'Stoch_d',
                              'OBV', 'Upper_BB', 'Middle_BB', 'Lower_BB', 'ATR_1', 'ADX', '+DI', '-DI', 'CCI']
        self.reset()

    def reset(self):
        self.current_step = 0
        self.balance = INITIAL_CASH
        self.shares_held = 0
        self.total_shares_traded = 0
        self.cumulative_reward = 0
        self.trades = []
        self.portfolio = {'cash': self.balance, 'holdings': {ticker: 0 for ticker in self.data['symbol'].unique()}}
        self.data['RSI'] = calculate_rsi(self.data['price'])
        self.data['pct_change'] = self.data['price'].pct_change()
        self.data['rolling_mean_vol'], self.data['rolling_std_vol'], self.data['rolling_mean_liq'], self.data['rolling_std_liq'] = calculate_vol_and_liquidity(self.data['price'], self.data['liquidity'], self.window_size)

    def step(self):
        row = self.data.iloc[self.current_step]
        current_price = row['price']
        current_time = pd.to_datetime(row['ts_event'])
        current_rsi = row['RSI']
        current_vol = row['pct_change']
        current_liq = row['liquidity']
        mean_vol = row['rolling_mean_vol']
        std_vol = row['rolling_std_vol']
        mean_liq = row['rolling_mean_liq']
        std_liq = row['rolling_std_liq']

        if current_rsi < 30:  # Entry signal based on RSI
            trade_direction = 'BUY'
            trade_price = get_trade_price(current_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction)
            trade_size = (self.portfolio['cash'] * np.random.uniform(0.001, 0.005)) / trade_price
            if self.portfolio['cash'] >= trade_size * trade_price:
                self.portfolio['cash'] -= trade_size * trade_price
                self.portfolio['holdings'][row['symbol']] += trade_size
                trade_status = 'filled'
            else:
                trade_status = 'cancelled'
        elif current_rsi > 70:  # Exit signal based on RSI
            trade_direction = 'SELL'
            if self.portfolio['holdings'][row['symbol']] > 0:
                trade_size = min(self.portfolio['holdings'][row['symbol']], self.portfolio['cash']*np.random.uniform(0.001, 0.005) / current_price)
                trade_price = get_trade_price(current_price, current_vol, current_liq, mean_vol, std_vol, mean_liq, std_liq, trade_direction)
                self.portfolio['cash'] += trade_size * trade_price
                self.portfolio['holdings'][row['symbol']] -= trade_size
                trade_status = 'filled'
            else:
                trade_size = 0
                trade_status = 'cancelled'
        else:
            trade_direction = 'HOLD'
            trade_size = 0
            trade_price = current_price
            trade_status = 'skipped'

        if trade_size > 0:
            expected_price = row['ask_px_00']
            actual_price = row['price']
            transaction_time = row['ts_in_delta']
            transaction_cost = self._calculate_transaction_cost(row['Volume'], 0.3, self.data['Volume'].mean())
            slippage = expected_price - actual_price
            time_penalty = 1000 * transaction_time / 1e9
            reward = - (slippage + time_penalty + transaction_cost)

            self.cumulative_reward += reward
            self.trades.append({
                'step': self.current_step,
                'timestamp': current_time,
                'action': trade_direction,
                'price': trade_price,
                'shares': trade_size,
                'symbol': row['symbol'],
                'reward': reward,
                'transaction_cost': transaction_cost,
                'slippage': slippage,
                'time_penalty': time_penalty
            })



        self.current_step += 1
        if self.current_step >= len(self.data) - 1:
            done=True
            self.current_step = 0

    def _calculate_transaction_cost(self, volume, volatility, daily_volume):
        return volatility * np.sqrt(volume / daily_volume)

    def run(self):
        self.reset()
        for _ in range(len(self.data)):
            self.step()
        return self.cumulative_reward, self.trades

    def render(self):
        print(f'Cumulative reward: {self.cumulative_reward}')
        row = self.data.iloc[self.current_step]
        print(f'Total portfolio value: {self.portfolio["cash"] + self.portfolio["holdings"][row["symbol"]]*row["Close"]}')
        # get trades in a pandas dataframe
        trades_df = pd.DataFrame(self.trades)
        # Save a csv
        trades_df.to_csv('trades_blotter.csv', index=False)
        for trade in self.trades:
            print(f"Step: {trade['step']}, Timestamp: {trade['timestamp']}, Action: {trade['action']}, Price: {trade['price']}, Shares: {trade['shares']}, Symbol: {trade['symbol']}, Reward: {trade['reward']}, Transaction Cost: {trade['transaction_cost']}, Slippage: {trade['slippage']}, Time Penalty: {trade['time_penalty']}")

#### Run the trading blotter

In [19]:
# Filter data for the specified ticker
ticker = 'AAPL'  # Specify the ticker you want to trade
ticker_data = market_features_df[market_features_df['symbol'] == ticker]

window_size = 60
daily_trading_limit = 1000
# Create the trading environment
env = TradingEnvironmentwithBlotter(ticker_data, daily_trading_limit=1000, window_size=window_size)  # Daily trading limit of 1000 shares

# Run the environment
cumulative_reward, trades = env.run()

# Render the results
env.render()

Streaming output truncated to the last 5000 lines.
Step: 49815, Timestamp: 2023-07-03 16:29:54.454344746, Action: SELL, Price: 182.7244085898357, Shares: 0.2578531399015479, Symbol: AAPL, Reward: -0.25111421803554473, Transaction Cost: 0.08031621803554471, Slippage: 0.0, Time Penalty: 0.170798
Step: 49816, Timestamp: 2023-07-03 16:29:54.454344746, Action: SELL, Price: 190.33458347928917, Shares: 0.1615208473104083, Symbol: AAPL, Reward: -0.19619621820430158, Transaction Cost: 0.025398218204301576, Slippage: 0.0, Time Penalty: 0.170798
Step: 49817, Timestamp: 2023-07-03 16:29:54.454344746, Action: SELL, Price: 189.22520793688514, Shares: 0.2623046761034777, Symbol: AAPL, Reward: -0.42478018204301576, Transaction Cost: 0.25398218204301576, Slippage: 0.0, Time Penalty: 0.170798
Step: 49818, Timestamp: 2023-07-03 16:29:54.474421200, Action: SELL, Price: 197.52567198100456, Shares: 0.20679496408808612, Symbol: AAPL, Reward: -0.4312691820430067, Transaction Cost: 0.25398218204301576, Slippag

In [20]:
df=market_features_df.copy()

In [21]:
df['timestamp']=pd.to_datetime(df['ts_recv'])

In [22]:
df.head()

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,Close,Volume,High,Low,Open,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ATR_2,ATR_5,ATR_10,ATR_20,ADX,+DI,-DI,CCI,DLR,TWAP,VWAP,timestamp
35,1688371214386057385,1688371214385893078,1,2,32,T,N,0,194.05,50,130,164307,326232,194.0,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.0,194.05,54.544543,0.006271,-0.003130,0.009401,52.525253,61.952862,-266.0,194.065621,194.0170,193.968379,0.30,0.175078,0.098615,0.075141,0.072403,97.257397,30.435801,0.196362,166.666667,0.000000,194.020000,194.021894,2023-07-03 08:00:14.386057385
36,1688371214386063777,1688371214385899379,1,2,32,T,N,0,194.05,50,130,164398,326233,194.0,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.0,194.05,54.544543,0.007108,-0.001082,0.008190,38.383838,52.525253,-266.0,194.068990,194.0200,193.971010,0.30,0.237539,0.138892,0.097627,0.083783,97.361721,22.989295,0.148320,83.333333,0.000000,194.020811,194.025188,2023-07-03 08:00:14.386063777
37,1688371215804852019,1688371215804687301,1,2,32,T,B,0,194.21,10,130,164718,328131,194.0,194.21,3101,29,4,1,AAPL,194.21,10,194.21,194.0,194.05,85.890753,0.020446,0.003223,0.017223,40.404040,43.771044,-256.0,194.125889,194.0305,193.935111,0.21,0.223770,0.153114,0.108864,0.090094,97.458593,19.409454,0.125224,79.268293,0.000824,194.025789,194.025596,2023-07-03 08:00:15.804852019
38,1688371219671476629,1688371219671312224,1,2,32,T,N,0,194.14,10,130,164405,331406,194.0,194.16,3101,400,4,1,AAPL,194.14,10,194.16,194.0,194.21,64.827662,0.025079,0.007594,0.017484,49.494949,42.760943,-266.0,194.142928,194.0375,193.932072,0.21,0.216885,0.164491,0.118978,0.096089,97.548546,16.622008,0.107240,-3.205128,-0.000360,194.028718,194.025873,2023-07-03 08:00:19.671476629
39,1688371223368835585,1688371223368671235,1,2,32,T,B,0,194.13,10,130,164350,334235,194.0,194.13,3101,400,4,1,AAPL,194.13,10,194.13,194.0,194.14,62.470772,0.027625,0.011601,0.016025,57.575758,49.158249,-276.0,194.155247,194.0440,193.932753,0.14,0.178442,0.159593,0.121080,0.098285,97.632074,15.068361,0.097216,-113.095238,-0.000052,194.031250,194.026071,2023-07-03 08:00:23.368835585


In [23]:
cumulative_reward

-12231.775689639546

## Transformer Implementation

References:
1. https://medium.com/@Matthew_Frank/stock-price-prediction-using-transformers-2d84341ff213
2. https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb


In [34]:
df_with_indicators
#market_features_df = df_with_indicators[35:]

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,Close,Volume,High,Low,Open,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ATR_2,ATR_5,ATR_10,ATR_20,ADX,+DI,-DI,CCI,DLR,TWAP,VWAP
0,1688371200660869841,1688371200660704717,1,2,32,T,B,0,194.12,1,130,165124,303634,193.63,194.12,27,27,1,1,AAPL,194.12,1,194.12,193.63,194.12,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.120000,193.875000
1,1688371201201402566,1688371201201237816,1,2,32,T,B,0,194.11,2,130,164750,304724,193.90,194.11,5,400,1,1,AAPL,194.11,2,194.11,193.90,194.12,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000052,194.115000,193.961667
2,1688371201233688992,1688371201233524761,1,2,32,T,B,0,194.11,8,130,164231,304850,193.90,194.11,5,398,1,1,AAPL,194.11,8,194.11,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.21,0.215000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,194.113333,193.993182
3,1688371201317556361,1688371201317392163,1,2,32,T,B,0,194.11,2,130,164198,305101,193.90,194.11,5,390,1,1,AAPL,194.11,2,194.11,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.21,0.212500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,194.112500,193.995000
4,1688371201478520666,1688371201478356044,1,2,32,T,B,0,194.00,7,130,164622,306430,193.90,194.00,5,200,1,1,AAPL,194.00,7,194.00,193.90,194.11,NaN,NaN,NaN,NaN,NaN,NaN,-8.0,NaN,NaN,NaN,0.21,0.211250,NaN,NaN,NaN,NaN,NaN,NaN,-74.468085,-0.000567,194.090000,193.979250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,1688417954514485218,1688417954514320323,1,2,32,T,B,0,192.44,6,130,164895,252532002,192.40,192.44,40,7,1,1,AAPL,192.44,6,192.44,192.40,192.40,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,0.034997,0.031258,0.028636,0.027592,15.474528,6.612534,3.765166,118.055556,0.000208,192.701135,192.722211
59267,1688417961020718430,1688417961020553920,1,2,32,T,B,0,192.44,1,130,164510,252532102,192.40,192.44,40,1,1,1,AAPL,192.44,1,192.44,192.40,192.44,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,0.037498,0.033006,0.029772,0.028212,16.329018,5.953254,3.389772,83.333333,0.000000,192.701131,192.722211
59268,1688417973297905504,1688417973297741235,1,2,32,T,A,0,192.40,5,130,164269,252532347,192.40,192.46,40,6,1,1,AAPL,192.40,5,192.46,192.40,192.44,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,0.048749,0.038405,0.032795,0.029802,19.013869,9.751295,2.919558,12.820513,-0.000208,192.701125,192.722211
59269,1688417996889779362,1688417996889614660,1,2,32,T,B,0,192.45,3,130,164702,252532944,192.40,192.45,35,16,1,1,AAPL,192.45,3,192.45,192.40,192.40,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,0.049375,0.040724,0.034515,0.030812,21.506945,8.671762,2.596344,100.000000,0.000260,192.701121,192.722211


In [35]:
market_features_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,symbol,Close,Volume,High,Low,Open,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ATR_2,ATR_5,ATR_10,ATR_20,ADX,+DI,-DI,CCI,DLR,TWAP,VWAP
35,1688371214386057385,1688371214385893078,1,2,32,T,N,0,194.05,50,130,164307,326232,194.0,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.0,194.05,54.544543,0.006271,-0.003130,0.009401,52.525253,61.952862,-266.0,194.065621,194.0170,193.968379,0.30,0.175078,0.098615,0.075141,0.072403,97.257397,30.435801,0.196362,166.666667,0.000000,194.020000,194.021894
36,1688371214386063777,1688371214385899379,1,2,32,T,N,0,194.05,50,130,164398,326233,194.0,194.30,3101,19,4,10,AAPL,194.05,50,194.30,194.0,194.05,54.544543,0.007108,-0.001082,0.008190,38.383838,52.525253,-266.0,194.068990,194.0200,193.971010,0.30,0.237539,0.138892,0.097627,0.083783,97.361721,22.989295,0.148320,83.333333,0.000000,194.020811,194.025188
37,1688371215804852019,1688371215804687301,1,2,32,T,B,0,194.21,10,130,164718,328131,194.0,194.21,3101,29,4,1,AAPL,194.21,10,194.21,194.0,194.05,85.890753,0.020446,0.003223,0.017223,40.404040,43.771044,-256.0,194.125889,194.0305,193.935111,0.21,0.223770,0.153114,0.108864,0.090094,97.458593,19.409454,0.125224,79.268293,0.000824,194.025789,194.025596
38,1688371219671476629,1688371219671312224,1,2,32,T,N,0,194.14,10,130,164405,331406,194.0,194.16,3101,400,4,1,AAPL,194.14,10,194.16,194.0,194.21,64.827662,0.025079,0.007594,0.017484,49.494949,42.760943,-266.0,194.142928,194.0375,193.932072,0.21,0.216885,0.164491,0.118978,0.096089,97.548546,16.622008,0.107240,-3.205128,-0.000360,194.028718,194.025873
39,1688371223368835585,1688371223368671235,1,2,32,T,B,0,194.13,10,130,164350,334235,194.0,194.13,3101,400,4,1,AAPL,194.13,10,194.13,194.0,194.14,62.470772,0.027625,0.011601,0.016025,57.575758,49.158249,-276.0,194.155247,194.0440,193.932753,0.14,0.178442,0.159593,0.121080,0.098285,97.632074,15.068361,0.097216,-113.095238,-0.000052,194.031250,194.026071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,1688417954514485218,1688417954514320323,1,2,32,T,B,0,192.44,6,130,164895,252532002,192.4,192.44,40,7,1,1,AAPL,192.44,6,192.44,192.4,192.40,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,0.034997,0.031258,0.028636,0.027592,15.474528,6.612534,3.765166,118.055556,0.000208,192.701135,192.722211
59267,1688417961020718430,1688417961020553920,1,2,32,T,B,0,192.44,1,130,164510,252532102,192.4,192.44,40,1,1,1,AAPL,192.44,1,192.44,192.4,192.44,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,0.037498,0.033006,0.029772,0.028212,16.329018,5.953254,3.389772,83.333333,0.000000,192.701131,192.722211
59268,1688417973297905504,1688417973297741235,1,2,32,T,A,0,192.40,5,130,164269,252532347,192.4,192.46,40,6,1,1,AAPL,192.40,5,192.46,192.4,192.44,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,0.048749,0.038405,0.032795,0.029802,19.013869,9.751295,2.919558,12.820513,-0.000208,192.701125,192.722211
59269,1688417996889779362,1688417996889614660,1,2,32,T,B,0,192.45,3,130,164702,252532944,192.4,192.45,35,16,1,1,AAPL,192.45,3,192.45,192.4,192.40,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,0.049375,0.040724,0.034515,0.030812,21.506945,8.671762,2.596344,100.000000,0.000260,192.701121,192.722211


In [114]:
state_columns = ['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'Stoch_k', 'Stoch_d',
                              'OBV', 'Upper_BB', 'Middle_BB', 'Lower_BB', 'ATR_1', 'ADX', '+DI', '-DI', 'CCI']
market_features_df_new = market_features_df[state_columns + ['Close']]

In [48]:
import math
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D

In [49]:
def calculate_bollinger_bands(data, window=10, num_of_std=2):
    """Calculate Bollinger Bands"""
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return upper_band, lower_band

def calculate_rsi(data, window=10):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_roc(data, periods=10):
    """Calculate Rate of Change."""
    roc = ((data - data.shift(periods)) / data.shift(periods)) * 100
    return roc

In [51]:
ticker_data_frames = []
stats = {}
for ticker in ["AAPL"]:

    # Download historical data for the ticker
    #data = yf.download(ticker, period="ytd", interval="1d")
    data = df

    # Calculate the daily percentage change
    close = data['Close']
    #upper, lower = calculate_bollinger_bands(close, window=14, num_of_std=2)
    #width = upper - lower
    #rsi = calculate_rsi(close, window=14)
    rsi = data['RSI']
    #roc = calculate_roc(close, periods=14)
    volume = data['Volume']
    diff = data['Close'].diff(1)
    percent_change_close = data['Close'].pct_change() * 100

    # Create a DataFrame for the current ticker and append it to the list
    ticker_df = pd.DataFrame({
        ticker+'_close': close,
        ticker+'_rsi': rsi,
        ticker+'_volume': volume,
        ticker+'_diff': diff,
        ticker+'_percent_change_close': percent_change_close,
    })

    MEAN = ticker_df.mean()
    STD = ticker_df.std()

    # Keep track of mean and std
    for column in MEAN.index:
      stats[f"{column}_mean"] = MEAN[column]
      stats[f"{column}_std"] = STD[column]

    # Normalize the training features
    ticker_df = (ticker_df - MEAN) / STD

    ticker_data_frames.append(ticker_df)

In [52]:
# Convert the dictionary containing feature statistics to a DataFrame for easier access
stats = pd.DataFrame([stats], index=[0])

# Display the DataFrame to verify its structure
stats.head()

,AAPL_close_mean,AAPL_close_std,AAPL_rsi_mean,AAPL_rsi_std,AAPL_volume_mean,AAPL_volume_std,AAPL_diff_mean,AAPL_diff_std,AAPL_percent_change_close_mean,AAPL_percent_change_close_std
0,192.700338,0.562008,47.270843,25.859923,139.519853,10402.161796,-0.000027,0.007325,-0.000014,0.003793


In [60]:
df = pd.concat(ticker_data_frames, axis=1)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df.head()

,AAPL_close,AAPL_rsi,AAPL_volume,AAPL_diff,AAPL_percent_change_close
36,2.401499,0.281273,-0.008606,0.003688,0.003666
37,2.686193,1.493427,-0.012451,21.847311,21.740965
38,2.561639,0.678920,-0.012451,-9.552898,-9.498568
39,2.543846,0.587779,-0.012451,-1.361539,-1.354285
40,2.543846,0.587779,-0.003799,0.003688,0.003666


In [61]:

# Shift the dataframe up by one to align current features with the next step's outcomes
labels = df.shift(-1)

# Remove the last row from both the features and labels to maintain consistent data pairs
df = df.iloc[:-1]
labels = labels.iloc[:-1]

In [62]:
labels

,AAPL_close,AAPL_rsi,AAPL_volume,AAPL_diff,AAPL_percent_change_close
36,2.686193,1.493427,-0.012451,21.847311,21.740965
37,2.561639,0.678920,-0.012451,-9.552898,-9.498568
38,2.543846,0.587779,-0.012451,-1.361539,-1.354285
39,2.543846,0.587779,-0.003799,0.003688,0.003666
40,2.348119,-0.177651,-0.012451,-15.013804,-14.934569
...,...,...,...,...,...
59265,-0.463229,0.610635,-0.012836,5.464594,5.484595
59266,-0.463229,0.610635,-0.013316,0.003688,0.003666
59267,-0.534402,-0.107184,-0.012932,-5.457218,-5.476124
59268,-0.445435,0.501904,-0.013124,6.829820,6.854827


In [63]:
SEQUENCE_LEN = 5

def create_sequences(data, labels, mean, std, sequence_length=SEQUENCE_LEN):
    sequences = []
    lab = []
    data_size = len(data)

    # Loop to create each sequence and its corresponding label
    for i in range(data_size - (sequence_length + 13)): # Ensure we have data for the label
        if i == 0:
          continue
        sequences.append(data[i:i + sequence_length])  # The sequence of data
        lab.append([labels[i-1], labels[i + 12], mean[0], std[0]]) # The label and scaling factors

    return np.array(sequences), np.array(lab)

In [64]:
df

,AAPL_close,AAPL_rsi,AAPL_volume,AAPL_diff,AAPL_percent_change_close
36,2.401499,0.281273,-0.008606,0.003688,0.003666
37,2.686193,1.493427,-0.012451,21.847311,21.740965
38,2.561639,0.678920,-0.012451,-9.552898,-9.498568
39,2.543846,0.587779,-0.012451,-1.361539,-1.354285
40,2.543846,0.587779,-0.003799,0.003688,0.003666
...,...,...,...,...,...
59265,-0.534402,-0.191737,-0.011971,0.003688,0.003666
59266,-0.463229,0.610635,-0.012836,5.464594,5.484595
59267,-0.463229,0.610635,-0.013316,0.003688,0.003666
59268,-0.534402,-0.107184,-0.012932,-5.457218,-5.476124


In [66]:
sequences_dict = {}
sequence_labels = {}
for ticker in ["AAPL"]:

    '''
    # Extract close and volume data for the ticker
    close = df[ticker+'_close'].values
    width = df[ticker+'_width'].values
    rsi = df[ticker+'_rsi'].values
    roc = df[ticker+'_roc'].values
    volume = df[ticker+'_volume'].values
    diff = df[ticker+'_diff'].values
    pct_change = df[ticker+'_percent_change_close'].values
    '''

    close = df[ticker + '_close'].values
    rsi = df[ticker+'_rsi'].values
    volume = df[ticker+'_volume'].values
    diff = df[ticker+'_diff'].values
    pct_change = df[ticker+'_percent_change_close'].values

    # Combine close and volume data
    ticker_data = np.column_stack((close,
                                   rsi,
                                   volume,
                                   diff,
                                   pct_change))

    # Generate sequences
    attribute = ticker+"_close"
    ticker_sequences, lab = create_sequences(ticker_data,
                                             labels[attribute].values[SEQUENCE_LEN-1:],
                                             stats[attribute+"_mean"].values,
                                             stats[attribute+"_std"].values)

    sequences_dict[ticker] = ticker_sequences
    sequence_labels[ticker] = lab

In [67]:
# Combine data and labels from all tickers
all_sequences = []
all_labels = []
tickers = ["AAPL"]
for ticker in tickers:
    all_sequences.extend(sequences_dict[ticker])
    all_labels.extend(sequence_labels[ticker])

# Convert to numpy arrays
all_sequences = np.array(all_sequences)
all_labels = np.array(all_labels)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [68]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Add()([x, inputs])

    # Feed Forward Part
    y = LayerNormalization(epsilon=1e-6)(x)
    y = Dense(ff_dim, activation="relu")(y)
    y = Dropout(dropout)(y)
    y = Dense(inputs.shape[-1])(y)
    return Add()([y, x])

In [69]:
def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout=0):
    inputs = Input(shape=input_shape)
    x = inputs
    for _ in range(num_layers):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    x = GlobalAveragePooling1D()(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    outputs = Dense(1, activation="linear")(x)
    return Model(inputs=inputs, outputs=outputs)

In [70]:
all_sequences.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(59216, 5, 5)

In [75]:
np.random.seed(42)
shuffled_indices = np.random.permutation(len(all_sequences))
all_sequences = all_sequences[shuffled_indices]
all_labels = all_labels[shuffled_indices]

train_size = int(len(all_sequences) * 0.9)

# Split sequences
train_sequences = all_sequences[:train_size]
train_labels    = all_labels[:train_size]

other_sequences = all_sequences[train_size:]
other_labels    = all_labels[train_size:]

shuffled_indices = np.random.permutation(len(other_sequences))
other_sequences = other_sequences[shuffled_indices]
other_labels = other_labels[shuffled_indices]

val_size = int(len(other_sequences) * 0.5)

validation_sequences = other_sequences[:val_size]
validation_labels = other_labels[:val_size]

test_sequences = other_sequences[val_size:]
test_labels = other_labels[val_size:]

In [76]:
input_shape = train_sequences.shape[1:]
head_size = 256
num_heads = 16
ff_dim = 1024
num_layers = 12
dropout = 0.20

model = build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout)
model.summary()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 5, 5)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_25    │ (None, 5, 5)           │             10 │ input_layer_1[0][0]    │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_12   │ (None, 5, 5)           │         94,213 │ layer_normalization_2… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_24 (Add)              │ (None, 5, 5)           │              0 │ multi_head_attention_… │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_26    │ (None, 5, 5)           │             10 │ add_24[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_25 (Dense)          │ (None, 5, 1024)        │          6,144 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_25 (Dropout)      │ (None, 5, 1024)        │              0 │ dense_25[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_26 (Dense)          │ (None, 5, 5)           │          5,125 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_25 (Add)              │ (None, 5, 5)           │              0 │ dense_26[0][0],        │
│                           │                        │                │ add_24[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_27    │ (None, 5, 5)           │             10 │ add_25[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_13   │ (None, 5, 5)           │         94,213 │ layer_normalization_2… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_26 (Add)              │ (None, 5, 5)           │              0 │ multi_head_attention_… │
│                           │                        │                │ add_25[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_28    │ (None, 5, 5)           │             10 │ add_26[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_27 (Dense)          │ (None, 5, 1024)        │          6,144 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_27 (Dropout) 

 Total params: 1,266,040 (4.83 MB)

 Trainable params: 1,266,040 (4.83 MB)

 Non-trainable params: 0 (0.00 B)

In [77]:
def custom_mae_loss(y_true, y_pred):
    y_true_next = tf.cast(y_true[:, 1], tf.float64)  # Extract the true next values, scaled
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)  # Extract the predicted next values, scaled
    abs_error = tf.abs(y_true_next - y_pred_next)  # Calculate the absolute error
    return tf.reduce_mean(abs_error)  # Return the mean of these errors

def dir_acc(y_true, y_pred):
    mean, std = tf.cast(y_true[:, 2], tf.float64), tf.cast(y_true[:, 3], tf.float64)  # Retrieve scaling factors
    y_true_prev = (tf.cast(y_true[:, 0], tf.float64) * std) + mean  # Un-scale previous true price
    y_true_next = (tf.cast(y_true[:, 1], tf.float64) * std) + mean  # Un-scale next true price
    y_pred_next = (tf.cast(y_pred[:, 0], tf.float64) * std) + mean  # Un-scale predicted next price

    true_change = y_true_next - y_true_prev  # Calculate true change
    pred_change = y_pred_next - y_true_prev  # Calculate predicted change

    correct_direction = tf.equal(tf.sign(true_change), tf.sign(pred_change))  # Check if the signs match
    return tf.reduce_mean(tf.cast(correct_direction, tf.float64))  # Return the mean of correct directions

In [78]:
# Define a callback to save the best model
checkpoint_callback_train = ModelCheckpoint(
    "transformer_train_model.keras",  # Filepath to save the best model
    monitor="dir_acc",  #"loss",  # Metric to monitor
    save_best_only=True,  # Save only the best model
    mode="max",  # Minimize the monitored metric
    verbose=1,  # Display progress
)

# Define a callback to save the best model
checkpoint_callback_val = ModelCheckpoint(
    "transformer_val_model.keras",  # Filepath to save the best model
    monitor="val_dir_acc", #"val_loss",  # Metric to monitor
    save_best_only=True,  # Save only the best model
    mode="max",  # Minimize the monitored metric
    verbose=1,  # Display progress
)

def get_lr_callback(batch_size=16, mode='cos', epochs=500, plot=False):
    lr_start, lr_max, lr_min = 0.0001, 0.005, 0.00001  # Adjust learning rate boundaries
    lr_ramp_ep = int(0.30 * epochs)  # 30% of epochs for warm-up
    lr_sus_ep = max(0, int(0.10 * epochs) - lr_ramp_ep)  # Optional sustain phase, adjust as needed

    def lrfn(epoch):
        if epoch < lr_ramp_ep:  # Warm-up phase
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep:  # Sustain phase at max learning rate
            lr = lr_max
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        else:
            lr = lr_min  # Default to minimum learning rate if mode is not recognized

        return lr

    if plot:  # Plot learning rate curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.title('Learning Rate Scheduler')
        plt.show()

    return tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

In [79]:
model.compile(optimizer='rmsprop', loss='mse')
BATCH_SIZE = 64  # Number of training examples used to calculate each iteration's gradient
EPOCHS = 3  # Total number of times the entire dataset is passed through the network

model.fit(
    train_sequences,  # Training features
    train_labels,  # Training labels
    validation_data=(validation_sequences, validation_labels),  # Validation data
    epochs=EPOCHS,  # Number of epochs to train for
    batch_size=BATCH_SIZE,  # Size of each batch
    shuffle=True,  # Shuffle training data before each epoch
    callbacks=[checkpoint_callback_train, checkpoint_callback_val, get_lr_callback(batch_size=BATCH_SIZE, epochs=EPOCHS)]  # Callbacks for saving models and adjusting learning rate
)



Epoch 1: LearningRateScheduler setting learning rate to 0.005.
Epoch 1/3
833/833 ━━━━━━━━━━━━━━━━━━━━ 1150s 1s/step - loss: 7808.4756 - val_loss: 6948.7310 - learning_rate: 0.0050

Epoch 2: LearningRateScheduler setting learning rate to 0.0037525.
Epoch 2/3


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with dir_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_dir_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


833/833 ━━━━━━━━━━━━━━━━━━━━ 1153s 1s/step - loss: 6949.4902 - val_loss: 6948.6821 - learning_rate: 0.0038

Epoch 3: LearningRateScheduler setting learning rate to 0.0012575000000000008.
Epoch 3/3
833/833 ━━━━━━━━━━━━━━━━━━━━ 1184s 1s/step - loss: 6949.5200 - val_loss: 6948.6714 - learning_rate: 0.0013


In [81]:
test_labels_predicted = model.predict(test_sequences)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


93/93 ━━━━━━━━━━━━━━━━━━━━ 36s 387ms/step


In [82]:
test_mse = np.mean(np.square((test_labels_predicted - test_labels)))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [83]:
test_mse

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


6948.045454412085

In [84]:
test_labels_predicted

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[47.966827],
       [48.949627],
       [47.886997],
       ...,
       [48.987484],
       [48.612698],
       [47.78423 ]], dtype=float32)

In [85]:
train_labels_predicted = model.predict(train_sequences)
train_mse = np.mean(np.square((train_labels_predicted - train_labels)))

1666/1666 ━━━━━━━━━━━━━━━━━━━━ 357s 214ms/step


In [86]:
train_mse

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


6949.449237325677

In [87]:
train_labels_predicted

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[48.516838],
       [47.9464  ],
       [47.74595 ],
       ...,
       [48.849766],
       [48.899067],
       [48.685436]], dtype=float32)

In [89]:
train_labels

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[  0.44423172,   0.40864504, 192.70033822,   0.56200799],
       [ -0.7657155 ,  -0.71233547, 192.70033822,   0.56200799],
       [ -1.19275569,  -1.17496235, 192.70033822,   0.56200799],
       ...,
       [  0.24850497,   0.25740164, 192.70033822,   0.56200799],
       [  0.61326847,   0.62216514, 192.70033822,   0.56200799],
       [  0.44423172,   0.40864504, 192.70033822,   0.56200799]])

In [90]:
train_labels.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(53294, 4)

In [91]:
train_labels_predicted*4

array([[194.06735],
       [191.7856 ],
       [190.9838 ],
       ...,
       [195.39906],
       [195.59627],
       [194.74174]], dtype=float32)

In [94]:
train_labels_predicted = [x[0]*4 for x in train_labels_predicted]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [95]:
train_labels_predicted

[194.06735229492188,
 191.7855987548828,
 190.98379516601562,
 191.19627380371094,
 191.337646484375,
 191.23558044433594,
 191.1577911376953,
 190.87742614746094,
 191.87635803222656,
 195.60357666015625,
 194.80101013183594,
 191.99546813964844,
 191.0493927001953,
 194.4208526611328,
 193.9903106689453,
 190.86265563964844,
 196.02589416503906,
 194.6083221435547,
 196.02430725097656,
 192.1614990234375,
 191.47108459472656,
 194.19667053222656,
 195.65126037597656,
 195.4875946044922,
 191.9875946044922,
 191.72642517089844,
 191.76365661621094,
 195.29981994628906,
 191.0279083251953,
 190.83653259277344,
 195.16607666015625,
 195.79579162597656,
 191.181396484375,
 194.59861755371094,
 191.67662048339844,
 194.12339782714844,
 190.80844116210938,
 191.11297607421875,
 191.63328552246094,
 191.8091583251953,
 192.2042694091797,
 191.3968048095703,
 194.43829345703125,
 191.1743621826172,
 195.8921661376953,
 195.9844512939453,
 191.06590270996094,
 191.56500244140625,
 195.4882202

In [92]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AAPL_close,AAPL_rsi,AAPL_volume,AAPL_diff,AAPL_percent_change_close
36,2.401499,0.281273,-0.008606,0.003688,0.003666
37,2.686193,1.493427,-0.012451,21.847311,21.740965
38,2.561639,0.678920,-0.012451,-9.552898,-9.498568
39,2.543846,0.587779,-0.012451,-1.361539,-1.354285
40,2.543846,0.587779,-0.003799,0.003688,0.003666
...,...,...,...,...,...
59266,-0.463229,0.610635,-0.012836,5.464594,5.484595
59267,-0.463229,0.610635,-0.013316,0.003688,0.003666
59268,-0.534402,-0.107184,-0.012932,-5.457218,-5.476124
59269,-0.445435,0.501904,-0.013124,6.829820,6.854827


In [93]:
market_features_df_new

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Volume,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ADX,+DI,-DI,CCI,Close
35,50,54.544543,0.006271,-0.003130,0.009401,52.525253,61.952862,-266.0,194.065621,194.0170,193.968379,0.30,97.257397,30.435801,0.196362,166.666667,194.05
36,50,54.544543,0.007108,-0.001082,0.008190,38.383838,52.525253,-266.0,194.068990,194.0200,193.971010,0.30,97.361721,22.989295,0.148320,83.333333,194.05
37,10,85.890753,0.020446,0.003223,0.017223,40.404040,43.771044,-256.0,194.125889,194.0305,193.935111,0.21,97.458593,19.409454,0.125224,79.268293,194.21
38,10,64.827662,0.025079,0.007594,0.017484,49.494949,42.760943,-266.0,194.142928,194.0375,193.932072,0.21,97.548546,16.622008,0.107240,-3.205128,194.14
39,10,62.470772,0.027625,0.011601,0.016025,57.575758,49.158249,-276.0,194.155247,194.0440,193.932753,0.14,97.632074,15.068361,0.097216,-113.095238,194.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,6,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,15.474528,6.612534,3.765166,118.055556,192.44
59267,1,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,16.329018,5.953254,3.389772,83.333333,192.44
59268,5,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,19.013869,9.751295,2.919558,12.820513,192.40
59269,3,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,21.506945,8.671762,2.596344,100.000000,192.45


In [101]:
market_features_df_new['Predicted Close Price'] = [0]*(6000-58) + list(train_labels_predicted)

<ipython-input-101-e2549c17e9ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_features_df_new['Predicted Close Price'] = [0]*(6000-58) + list(train_labels_predicted)


In [102]:
market_features_df_new

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Volume,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ADX,+DI,-DI,CCI,Close,Predicted Close Price
35,50,54.544543,0.006271,-0.003130,0.009401,52.525253,61.952862,-266.0,194.065621,194.0170,193.968379,0.30,97.257397,30.435801,0.196362,166.666667,194.05,0.000000
36,50,54.544543,0.007108,-0.001082,0.008190,38.383838,52.525253,-266.0,194.068990,194.0200,193.971010,0.30,97.361721,22.989295,0.148320,83.333333,194.05,0.000000
37,10,85.890753,0.020446,0.003223,0.017223,40.404040,43.771044,-256.0,194.125889,194.0305,193.935111,0.21,97.458593,19.409454,0.125224,79.268293,194.21,0.000000
38,10,64.827662,0.025079,0.007594,0.017484,49.494949,42.760943,-266.0,194.142928,194.0375,193.932072,0.21,97.548546,16.622008,0.107240,-3.205128,194.14,0.000000
39,10,62.470772,0.027625,0.011601,0.016025,57.575758,49.158249,-276.0,194.155247,194.0440,193.932753,0.14,97.632074,15.068361,0.097216,-113.095238,194.13,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,6,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,15.474528,6.612534,3.765166,118.055556,192.44,191.958130
59267,1,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,16.329018,5.953254,3.389772,83.333333,192.44,192.267395
59268,5,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,19.013869,9.751295,2.919558,12.820513,192.40,195.399063
59269,3,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,21.506945,8.671762,2.596344,100.000000,192.45,195.596268


In [103]:
market_features_df_new["Diff"] = market_features_df_new["Predicted Close Price"] - market_features_df_new["Close"]

<ipython-input-103-f5b2c13d4ba0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_features_df_new["Diff"] = market_features_df_new["Predicted Close Price"] - market_features_df_new["Close"]


In [104]:
market_features_df_new["action"] = market_features_df_new["Diff"].apply(lambda x: "BUY" if x > 0 else "SELL")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-104-4ead0359fc5b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_features_df_new["action"] = market_features_df_new["Diff"].apply(lambda x: "BUY" if x > 0 else "SELL")


In [105]:
market_features_df_new

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Volume,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ADX,+DI,-DI,CCI,Close,Predicted Close Price,Diff,action
35,50,54.544543,0.006271,-0.003130,0.009401,52.525253,61.952862,-266.0,194.065621,194.0170,193.968379,0.30,97.257397,30.435801,0.196362,166.666667,194.05,0.000000,-194.050000,SELL
36,50,54.544543,0.007108,-0.001082,0.008190,38.383838,52.525253,-266.0,194.068990,194.0200,193.971010,0.30,97.361721,22.989295,0.148320,83.333333,194.05,0.000000,-194.050000,SELL
37,10,85.890753,0.020446,0.003223,0.017223,40.404040,43.771044,-256.0,194.125889,194.0305,193.935111,0.21,97.458593,19.409454,0.125224,79.268293,194.21,0.000000,-194.210000,SELL
38,10,64.827662,0.025079,0.007594,0.017484,49.494949,42.760943,-266.0,194.142928,194.0375,193.932072,0.21,97.548546,16.622008,0.107240,-3.205128,194.14,0.000000,-194.140000,SELL
39,10,62.470772,0.027625,0.011601,0.016025,57.575758,49.158249,-276.0,194.155247,194.0440,193.932753,0.14,97.632074,15.068361,0.097216,-113.095238,194.13,0.000000,-194.130000,SELL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,6,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,15.474528,6.612534,3.765166,118.055556,192.44,191.958130,-0.481870,SELL
59267,1,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,16.329018,5.953254,3.389772,83.333333,192.44,192.267395,-0.172605,SELL
59268,5,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,19.013869,9.751295,2.919558,12.820513,192.40,195.399063,2.999063,BUY
59269,3,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,21.506945,8.671762,2.596344,100.000000,192.45,195.596268,3.146268,BUY


In [106]:
market_features_df_new = market_features_df_new[market_features_df_new["Predicted Close Price"] != 0]

In [109]:
market_features_df_new_shifted = market_features_df_new.copy()

market_features_df_new_shifted["Close"] = market_features_df_new_shifted["Close"].shift(-1)
#df['Salary'] = df['Salary'].shift(1)


In [110]:
market_features_df_new_shifted

,Volume,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ADX,+DI,-DI,CCI,Close,Predicted Close Price,Diff,action
5977,100,42.649072,-0.005775,-0.004481,-0.001294,11.111111,5.555556,874825.0,193.876396,193.8475,193.818604,0.01,32.773262,1.838238,4.103422,-83.333333,193.84,194.067352,0.227352,BUY
5978,2,42.649072,-0.005291,-0.004643,-0.000648,16.666667,11.111111,874825.0,193.873129,193.8460,193.818871,0.01,33.155440,1.798894,4.015596,-55.555556,193.84,191.785599,-2.054401,SELL
5979,200,42.649072,-0.004852,-0.004685,-0.000167,16.666667,14.814815,874825.0,193.869380,193.8445,193.819620,0.01,33.510320,1.758364,3.925123,-41.666667,193.85,190.983795,-2.856205,SELL
5980,49,56.650179,-0.003655,-0.004479,0.000824,22.222222,18.518519,874874.0,193.865658,193.8435,193.821342,0.02,35.162164,1.676986,6.057501,0.000000,193.83,191.196274,-2.653726,SELL
5981,393,37.127709,-0.004271,-0.004437,0.000167,16.666667,18.518519,874481.0,193.860761,193.8415,193.822239,0.02,36.696020,1.597372,5.769924,-166.666667,193.83,191.337646,-2.492354,SELL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59266,6,63.061829,0.005002,0.006368,-0.001366,33.333333,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,15.474528,6.612534,3.765166,118.055556,192.44,191.958130,-0.481870,SELL
59267,1,63.061829,0.006355,0.006365,-0.000011,66.666667,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,16.329018,5.953254,3.389772,83.333333,192.40,192.267395,-0.172605,SELL
59268,5,44.499081,0.004151,0.005922,-0.001771,66.666667,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,19.013869,9.751295,2.919558,12.820513,192.45,195.399063,2.999063,BUY
59269,3,60.250043,0.006366,0.006011,0.000355,61.111111,64.814815,913232.0,192.451394,192.4220,192.392606,0.05,21.506945,8.671762,2.596344,100.000000,192.45,195.596268,3.146268,BUY


In [111]:
market_features_df_new_shifted = market_features_df_new_shifted[:-1]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
market_features_df_new_shifted

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Volume,RSI,MACD,MACD_signal,MACD_hist,Stoch_k,Stoch_d,OBV,Upper_BB,Middle_BB,Lower_BB,ATR_1,ADX,+DI,-DI,CCI,Close,Predicted Close Price,Diff,action
5977,100,42.649072,-0.005775,-0.004481,-0.001294,1.111111e+01,5.555556,874825.0,193.876396,193.8475,193.818604,0.01,32.773262,1.838238,4.103422,-83.333333,193.84,194.067352,0.227352,BUY
5978,2,42.649072,-0.005291,-0.004643,-0.000648,1.666667e+01,11.111111,874825.0,193.873129,193.8460,193.818871,0.01,33.155440,1.798894,4.015596,-55.555556,193.84,191.785599,-2.054401,SELL
5979,200,42.649072,-0.004852,-0.004685,-0.000167,1.666667e+01,14.814815,874825.0,193.869380,193.8445,193.819620,0.01,33.510320,1.758364,3.925123,-41.666667,193.85,190.983795,-2.856205,SELL
5980,49,56.650179,-0.003655,-0.004479,0.000824,2.222222e+01,18.518519,874874.0,193.865658,193.8435,193.821342,0.02,35.162164,1.676986,6.057501,0.000000,193.83,191.196274,-2.653726,SELL
5981,393,37.127709,-0.004271,-0.004437,0.000167,1.666667e+01,18.518519,874481.0,193.860761,193.8415,193.822239,0.02,36.696020,1.597372,5.769924,-166.666667,193.83,191.337646,-2.492354,SELL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59265,15,42.312537,0.003202,0.006710,-0.003508,1.591616e-12,33.333333,913228.0,192.444483,192.4145,192.384517,0.03,14.554309,4.504976,4.196726,-55.555556,192.44,191.310211,-1.089789,SELL
59266,6,63.061829,0.005002,0.006368,-0.001366,3.333333e+01,22.222222,913234.0,192.444977,192.4175,192.390023,0.04,15.474528,6.612534,3.765166,118.055556,192.44,191.958130,-0.481870,SELL
59267,1,63.061829,0.006355,0.006365,-0.000011,6.666667e+01,33.333333,913234.0,192.447411,192.4195,192.391589,0.04,16.329018,5.953254,3.389772,83.333333,192.40,192.267395,-0.172605,SELL
59268,5,44.499081,0.004151,0.005922,-0.001771,6.666667e+01,55.555556,913229.0,192.447411,192.4195,192.391589,0.06,19.013869,9.751295,2.919558,12.820513,192.45,195.399063,2.999063,BUY


In [113]:
profit = 0
prev_close_price = market_features_df_new_shifted.iloc[0]["Close"]
for i in range(len(market_features_df_new_shifted)):
  r = market_features_df_new.iloc[i]
  curr_close_price = r["Close"]
  action = r["action"]

  if(action == "BUY"):
    profit += curr_close_price - prev_close_price

  else:
    profit += prev_close_price - curr_close_price


print("Total Profit = ", profit)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Total Profit =  334.34500000001844


In [115]:
market_features_df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['ts_recv', 'ts_event', 'rtype', 'publisher_id', 'instrument_id',
       'action', 'side', 'depth', 'price', 'size', 'flags', 'ts_in_delta',
       'sequence', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00',
       'bid_ct_00', 'ask_ct_00', 'symbol', 'Close', 'Volume', 'High', 'Low',
       'Open', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist', 'Stoch_k', 'Stoch_d',
       'OBV', 'Upper_BB', 'Middle_BB', 'Lower_BB', 'ATR_1', 'ATR_2', 'ATR_5',
       'ATR_10', 'ATR_20', 'ADX', '+DI', '-DI', 'CCI', 'DLR', 'TWAP', 'VWAP'],
      dtype='object')